# 1. Ocorrências aeronáuticas

# 2. Membros (nome e número de matrícula)
- Izabela Garcia (202206332)
- Isabella Carvalho ()
- Gabriel Vitor Gonçalves ()
- João Marcos Rezende ()

In [1]:
import csv
import pandas as pd
import sqlite3

import requests
from PIL import Image
from io import BytesIO
from collections import Counter

# 3. Descrição dos dados (qual a URL? qual o domínio? como os dados foram processados?)
## URL da base de dados

https://dados.gov.br/dados/conjuntos-dados/ocorrencias-aeronauticas

## Domínio dos dados

Este conjunto de dados contém informações sobre **ocorrências aeronáuticas**, incluindo acidentes e incidentes graves registrados pela Força Aérea Brasileira (FAB) e a ANAC. As ocorrências envolvem dados como a aeronave, o tipo de operação, o local, descrição do ocorrido, e as lesões causadas aos tripulantes, passageiros e terceiros.

### Objetivo do Dataset

O objetivo desse dataset é fornecer informações para análise e melhoria da **segurança operacional** na aviação, permitindo a investigação de incidentes e a identificação de padrões de risco. Ele é utilizado para monitorar a segurança, apoiar investigações e desenvolver estratégias para prevenção de acidentes.

## Pré-processamento dos dados
- Padronização dos valores nulos;
- Remoção de colunas "duplicadas" como *operador* e *operador_padronizado* mantendo apenas uma delas;
- Garantia de unicidade e existência dos valores que serão utilizados como ID;
- Renomear colunas para melhorar compreensão;
- Remoção de linhas com descrição nula, operador desconhecido, ou operação nulo;

In [2]:
source = 'V_OCORRENCIA_AMPLA.csv'
df = pd.read_csv(source, delimiter=';',quotechar='"', header=1)

# padronizando nulos
df.replace({'-': None, 'null': None, 'Indeterminado': None, '': None}, inplace=True)
df['Lesoes_Fatais_Tripulantes'] = df['Lesoes_Fatais_Tripulantes'].fillna('0')
df['Lesoes_Fatais_Passageiros'] = df['Lesoes_Fatais_Passageiros'].fillna('0')
df['Lesoes_Fatais_Terceiros'] = df['Lesoes_Fatais_Terceiros'].fillna('0')
df['Lesoes_Graves_Tripulantes'] = df['Lesoes_Graves_Tripulantes'].fillna('0')
df['Lesoes_Graves_Passageiros'] = df['Lesoes_Graves_Passageiros'].fillna('0')
df['Lesoes_Graves_Terceiros'] = df['Lesoes_Graves_Terceiros'].fillna('0')
df['Lesoes_Leves_Tripulantes'] = df['Lesoes_Graves_Tripulantes'].fillna('0')
df['Lesoes_Leves_Passageiros'] = df['Lesoes_Graves_Passageiros'].fillna('0')
df['Lesoes_Leves_Terceiros'] = df['Lesoes_Graves_Terceiros'].fillna('0')
df['Ilesos_Tripulantes'] = df['Ilesos_Tripulantes'].fillna('0')
df['Ilesos_Passageiros'] = df['Ilesos_Passageiros'].fillna('0')
df['Lesoes_Desconhecidas_Tripulantes'] = df['Lesoes_Desconhecidas_Tripulantes'].fillna('0')
df['Lesoes_Desconhecidas_Passageiros'] = df['Lesoes_Desconhecidas_Passageiros'].fillna('0')
df['Lesoes_Desconhecidas_Terceiros'] = df['Lesoes_Desconhecidas_Terceiros'].fillna('0')

# deletar coluna do operador
df.drop("Operador", axis=1, inplace=True)

# renomear colunas
df.columns = df.columns.str.strip()
df.rename(columns={'Historico': 'Descricao'}, inplace=True)
df.rename(columns={'Operador_Padronizado': 'Operador'}, inplace=True)
df.rename(columns={'Tipo_de_Ocorrencia': 'Tipo_ADREP'}, inplace=True)
df.rename(columns={'Descricao_do_Tipo':'Tipo_descricao'}, inplace=True)
df.rename(columns={'Tipo_ICAO':'Cod_ICAO'}, inplace=True)
print(len(df)) # tamanho original

# remoção de linhas com descrição nula
df = df.dropna(subset=['Descricao'])

# remoção de linhas com operador "DESCONHECIDO", "OPERADOR DESCONHECIDO" ou nulo
df = df.query(' Operador != "DESCONHECIDO" and Operador != "OPERADOR DESCONHECIDO" ')
df = df.dropna(subset=['Operador'])

# remoção de linhas com operação nula
df = df.dropna(subset=['Operacao'])

# remoção de linhas com aeronave com cod_ICAO nulo
df = df.dropna(subset=['Matricula'])

# remoção de linhas com Região nulos
df = df.dropna(subset=['Regiao'])

# remoção de duplicatas com mesmo número de ocorrência
df = df.drop_duplicates(subset=['Numero_da_Ocorrencia'], keep='first')

# checagem de ID
if df['Numero_da_Ocorrencia'].isna().sum() == 0 and df['Numero_da_Ocorrencia'].nunique() == len(df):
    print("Numero_da_Ocorrencia pode ser utilizado como ID")
else:
    print("Numero_da_Ocorrencia não pode ser utilizado como ID")

print(f'Tamanho do dataset: {len(df)}') # tamanho após remoções

print(df.info())

5021
Numero_da_Ocorrencia pode ser utilizado como ID
Tamanho do dataset: 3469
<class 'pandas.core.frame.DataFrame'>
Index: 3469 entries, 14 to 5014
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Numero_da_Ocorrencia              3469 non-null   int64  
 1   Numero_da_Ficha                   3469 non-null   object 
 2   Operador                          3469 non-null   object 
 3   Classificacao_da_Ocorrencia       3469 non-null   object 
 4   Data_da_Ocorrencia                3469 non-null   object 
 5   Hora_da_Ocorrencia                2852 non-null   object 
 6   Municipio                         2895 non-null   object 
 7   UF                                3469 non-null   object 
 8   Regiao                            3469 non-null   object 
 9   Tipo_descricao                    3399 non-null   object 
 10  ICAO                              1217 non-null   object 


### Tabela pré-processada

In [3]:
# Salvar o DataFrame em um novo arquivo CSV
output_csv = "ocorrencias_limpo.csv"
df.to_csv(output_csv, index=False, sep=';', encoding='utf-8')

### Criando o banco

In [4]:
src = 'ocorrencias_limpo.csv'

conn = sqlite3.connect('ocorrencias.db')
cur = conn.cursor()

cur.execute('PRAGMA synchronous = OFF')
cur.execute('PRAGMA journal_mode = MEMORY')

BATCH_SIZE = 1000

cur.execute('BEGIN TRANSACTION')

with open(src, 'r', newline='', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=';')
    header = next(reader)

    columns = [f'"{column}"' for column in header]
    create = f'CREATE TABLE IF NOT EXISTS Ocorrencias ({", ".join(columns)})'
    cur.execute(create)

    placeholders = ', '.join(['?'] * len(header))
    insert = f'INSERT INTO Ocorrencias VALUES ({placeholders})'

    batch = []
    for row in reader:
        row = [None if value == '' else value for value in row]
        batch.append(row)
        if len(batch) == BATCH_SIZE:
            cur.executemany(insert, batch)
            batch = []

    if batch:
        cur.executemany(insert, batch)

update = f'UPDATE Ocorrencias SET ' + ','.join([f'{column} = NULLIF({column}, "")' for column in columns])
cur.execute(update)

conn.commit()

cur.execute('PRAGMA synchronous = FULL')
cur.execute('PRAGMA journal_mode = DELETE')

## Compreensão dos dados

In [5]:
def fetch(query, conn, formatted=True):
    # execute the query and fetch all rows
    cur = conn.cursor()
    cur.execute(query)
    rs = cur.fetchall()
    
    # extract column names from the cursor description
    columns = [desc[0] for desc in cur.description]
    
    # return a dataframe with column names
    return pd.DataFrame(rs, columns=columns) if formatted else rs

def desc(table, conn):
    cur = conn.cursor()
    cur.execute(f'PRAGMA table_info("{table}")')
    columns = [row[1] for row in cur.fetchall()]
    
    return columns

def info(table, conn):
    df1 = fetch(f'PRAGMA table_info("{table}")', conn)
    columns = desc(table, conn)
    
    counts = ', '.join([f'COUNT(*) AS "{column}"' for column in columns])
    df2 = fetch(f'SELECT {counts} FROM "{table}"', conn).transpose()
    df2.columns = ['count']
    
    counts = ', '.join([f'COUNT("{column}") AS "{column}"' for column in columns])
    df3 = fetch(f'SELECT {counts} FROM "{table}"', conn).transpose()
    df3.columns = ['notnull count']

    counts = ', '.join([f'COUNT(DISTINCT "{column}") AS "{column}"' for column in columns])
    df4 = fetch(f'SELECT {counts} FROM "{table}"', conn).transpose()
    df4.columns = ['unique count']
    
    return df1.merge(df2, left_on='name', right_index=True) \
            .merge(df3, left_on='name', right_index=True) \
            .merge(df4, left_on='name', right_index=True)

In [6]:
info('Ocorrencias', conn)

,cid,name,type,notnull,dflt_value,pk,count,notnull count,unique count
0,0,Numero_da_Ocorrencia,,0,None,0,3469,3469,3469
1,1,Numero_da_Ficha,,0,None,0,3469,3469,3194
2,2,Operador,,0,None,0,3469,3469,2513
3,3,Classificacao_da_Ocorrencia,,0,None,0,3469,3469,2
4,4,Data_da_Ocorrencia,,0,None,0,3469,3469,2584
5,5,Hora_da_Ocorrencia,,0,None,0,3469,2852,681
6,6,Municipio,,0,None,0,3469,2895,1288
7,7,UF,,0,None,0,3469,3469,27
8,8,Regiao,,0,None,0,3469,3469,5
9,9,Tipo_descricao,,0,None,0,3469,3399,33


In [7]:
cur.close()
conn.close()

# 4. Diagrama ER

# 5. Diagrama relacional

# 6. Consultas

## 6.1 Duas consultas envolvendo seleção e projeção

### 6.1.1 Consulta 1

### 6.1.2 Consulta 2

## 6.2 Três consultas envolvendo junção de duas relações

### 6.2.1 Consulta 3

### 6.2.2 Consulta 4

### 6.2.3 Consulta 5

## 6.3 Três consultas envolvendo junção de três ou mais relações

### 6.3.1 Consulta 6

### 6.3.2 Consulta 7

### 6.3.3 Consulta 8

## 6.4 Duas consultas envolvendo agregação sobre junção de duas ou mais relações

### 6.4.1 Consulta 9

### 6.4.2 Consulta 10

# 7. Autoavaliação dos membros